In [1]:
import torch

# Local imports
from train import train_model

Using device: cpu


In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda


In [ ]:
args = {
    "use_small_dataset": False,
    "batch_size": 64,
    "stoppage_epochs": 512,
    "max_epochs": 512,
    "seed": 42,
    "data_path": "../data",
    "frac_train": 0.7,
    "frac_validation": 0.15,
    "frac_test": 0.15,
    "huber_beta": 1.0,
    "weight_decay": 1e-4,
    "lr": 1e-3,
    "momentum": 0.9,
    "scheduler_patience": 10,
    "scheduler_factor": 0.5,
    "hidden_size": 96,
    "num_layers": 4,
    "num_attn_heads": 6,
    "dropout": 0.0,
    "leaky_relu_slope": 0.2,
    "pooling_dim": 96,
}


In [ ]:
import argparse

training_args = argparse.Namespace(**args)
train_model(training_args, device)

Model parameters: 152467


Epoch 1/512:   6%|███▌                                                    | 50/796 [00:07<01:46,  6.99it/s]